In [ ]:
import ixmp
import pyam
import nomenclature

In [ ]:
ecemf_public = ixmp.Platform("ecemf")

In [ ]:
source_db = ixmp.Platform("ecemf-internal")

In [ ]:
scenarios = source_db.scenario_list()

In [ ]:
scenarios.model.unique()

In [ ]:
scenario_mapping = {
    "Euro-Calliope 2.0": [
        "DIAG-C400-lin",
    ],
    "LIMES 2.38": [
        "DIAG-C400-lin",
    ],
    "OSeMBE v1.0.0": [
        "DIAG-C400-lin",
    ],
    "MEESA v1.2": [
        "DIAG-C400-lin",
    ],
    "PRIMES 2022": [
        "DIAG-C400-lin",
    ],
    "REMIND 2.1": [
        "DIAG-C400-lin",
    ],
    "WITCH 5.1": [
        "DIAG-C400-lin",
    ],
    "IMAGE 3.2": [
        "DIAG-C400-lin",
    ],
    "MESSAGEix-GLOBIOM 1.2": [
        "DIAG-C400-lin",
    ],
    "TIAM-ECN 1.2": [
        "DIAG-C400-lin",
    ],
    "PROMETHEUS 1.2": [
        "DIAG-C400-lin",
    ],
}

In [ ]:
scenario_selection = list()
for model, scenarios in scenario_mapping.items():
    for s in scenarios:
        scenario_selection.append((model, s))

In [ ]:
scenario_selection

In [ ]:
scenario_list = [
    pyam.IamDataFrame(
        ixmp.Scenario(mp=source_db, model=m, scenario=s).timeseries()
    )
    for (m, s) in scenario_selection
]
df = pyam.concat(scenario_list)

In [ ]:
df.filter(year=range(1990, 2101, 1), inplace=True)

In [ ]:
for (_model, _scenario) in df.index:
    _df = df.filter(model=_model, scenario=_scenario)

    # initialize scenario and add timeseries data
    scen = ixmp.Scenario(ecemf_public, _model, _scenario, version="new")
    scen.add_timeseries(_df.data)
    scen.commit(f"Migrated from H2020 ECEMF project-internal database")
    scen.set_as_default()

In [ ]:
df.to_excel("ECEMF-Diagnostic-scenarios-v1.xlsx")

# Add diagnostic indicators

In [ ]:
dsd = nomenclature.DataStructureDefinition("../definitions/")

In [ ]:
countries = []

for country in dsd.region:
    if dsd.region[country].hierarchy == "country":
        if dsd.region[country].eu_member:
            countries.append(country)

In [ ]:
countries

In [ ]:
scenarios = ecemf_public.scenario_list()

In [ ]:
for i, row in scenarios.iterrows():
    run = ixmp.Scenario(ecemf_public, model=row["model"], scenario=row["scenario"])
    run.set_meta("doi", "https://doi.org/10.5281/zenodo.7634845")
    run.set_meta("citation", "Pietzcker et al., 2023, ZENODO")
    run.set_meta("Work package", "WP1")
    run.set_meta("Scenario type", "Diagnostic")
    
    df = run.timeseries()
    regions = list(df.region.unique())
    if "World" in regions:
        run.set_meta("coverage: global", "yes")
    if "OECD & EU (R5)" in regions:
        run.set_meta("coverage: R5", "yes")
    if "Europe (R10)" in regions:
        run.set_meta("coverage: R10", "yes")
    if "EU27 & UK" in regions:
        run.set_meta("coverage: EU27 & UK", "yes")
    if "EU27" in regions:
        run.set_meta("coverage: EU27", "yes")
    if set(regions).issubset(countries):
        run.set_meta("coverage: country", "yes")
